In [1]:
import pandas as pd
import os
import time
from IPython.display import display

In [2]:
#Global Variables
directory_CG = '/Users/jerenolsen/Desktop/Exploring PGP Harvard/Complete Genomics/all-data-by-profile'
directory_23 = '/Users/jerenolsen/Desktop/Exploring PGP Harvard/23andMe/all-data-by-profile'

In [3]:
#Generic Functions

def read_table(path):
    df = pd.read_csv(path, delimiter = '\t')
    
    return df

def get_filepaths(all_data_directory, subdir):
    # Returns all file paths in 'ehrs' or 'surveys' subdir
    if subdir == 'surveys':
        subdir= "/surveys"
    elif subdir == 'ehrs':
        subdir = "/ehrs"
    else:
        print("Enter correct subdir")
        return None
    
    directory = all_data_directory
    paths = []
    for filename in os.listdir(directory):
        path = os.path.join(directory, filename)
        if os.path.isdir(path+subdir):
            paths.append(path+subdir)
    return paths


In [4]:
# EHR Functions
def init_ehr_tables():
    demographics = pd.DataFrame(columns=['profile_id','Date of Birth', 'Gender', 'Weight', 'Height', 'Blood Type', 'Race'])
    conditions = pd.DataFrame(columns = ['Name', 'Start Date', 'End Date'])
    medications = pd.DataFrame(columns = ['Name', 'Dosage', 'Frequency', 'Start Date', 'End Date'])
    allergies = pd.DataFrame(columns = ['Name', 'Reaction/Severity', 'Start Date', 'End Date'])
    procedures = pd.DataFrame(columns = ['Name', 'Date'])
    test_results = pd.DataFrame(columns = ['Name', 'Result', 'Date'])
    immunizations = pd.DataFrame(columns = ['Name', 'Date'])
    
    ehr_tables = {'demographics': demographics,
                  'conditions': conditions,
                  'medications': medications,
                  'allergies': allergies,
                  'procedures': procedures,
                  'test_results': test_results,
                  'immunizations': immunizations}
    
    return ehr_tables


def process_table_type(ehr_dir_path, filename, ehr_tables):
    filepath = os.path.join(ehr_dir_path, filename)
    
    if "Demographic" in filename:
        process_table_demographics(ehr_dir_path, filepath, ehr_tables)
       
    elif 'Conditions' in filename:
        process_table_generic(ehr_dir_path, filepath, ehr_tables, tablename = 'conditions')
        
    elif 'Medications' in filename:
        process_table_generic(ehr_dir_path, filepath, ehr_tables, tablename = 'medications')
        
    elif 'Allergies' in filename:
        process_table_generic(ehr_dir_path, filepath, ehr_tables, tablename = 'allergies')
        
    elif 'Procedures' in filename:
        process_table_generic(ehr_dir_path, filepath, ehr_tables, tablename = 'procedures')
        
    elif 'Test Results' in filename:
        process_table_generic(ehr_dir_path, filepath, ehr_tables, tablename = 'test_results')
        
    elif 'Immunizations' in filename:
        process_table_generic(ehr_dir_path, filepath, ehr_tables, tablename = 'immunizations')
    
    else:
        pass
        
    return None

def read_and_update_tables(ehr_dir_path, ehr_tables):
    for file in os.listdir(ehr_dir_path):
        process_table_type(ehr_dir_path, file, ehr_tables)
        
    return None

    
#main function
def build_ehr_tables(all_data_directory):
    paths = get_filepaths(all_data_directory, 'ehrs')
    ehr_tables = init_ehr_tables()
    
    for path in paths:
        read_and_update_tables(path, ehr_tables)
        
    #Reorganize columns
    for table in ehr_tables:
        ehr_tables[table] = reorg_columns(ehr_tables[table])
        
    return ehr_tables
        
    

In [5]:
#Table Specific Functions
def transform_table_demographics(df):
    df = df.transpose()
    df = df.iloc[2:]
    df.columns = ['Date of Birth', 'Gender', 'Weight', 'Height', 'Blood Type', 'Race']
    df.reset_index(drop = True, inplace=True)
    return df

def process_table_demographics(ehr_dir_path, filepath, ehr_tables):
    df = read_table(filepath)
    df = transform_table_demographics(df)
    df = add_profileID_col(df, ehr_dir_path)
    df = add_provider_col(df, ehr_dir_path)
    ehr_tables['demographics'] = pd.concat([ehr_tables['demographics'], df])
    return None

def process_table_generic(ehr_dir_path, filepath, ehr_tables, tablename):
    df = read_table(filepath)
    df.drop(df.columns[0], axis = 1, inplace = True)
    df = add_profileID_col(df, ehr_dir_path)
    df = add_provider_col(df, ehr_dir_path)
    ehr_tables[tablename] = pd.concat([ehr_tables[tablename], df])
    return None

def add_profileID_col(df, path):
    profile_id = path[-13:-5]
    df.insert(0, 'profile_id', profile_id)
    return df

def add_provider_col(df, ehr_dir_path):
    provider_id = None
    if 'Complete' in ehr_dir_path:
        provider_id = 'CG'
    elif '23' in ehr_dir_path:
        provider_id = '23andMe'
        
    df.insert(1, 'sequence_provider', provider_id)
    return df

def reorg_columns(df):
    cols = list(df.columns.values)
    cols.pop(cols.index('profile_id'))
    cols.pop(cols.index('sequence_provider'))
    df = df[['profile_id','sequence_provider']+cols]
    df = df.reset_index(drop=True)
    return df


# Write Complete Genomics EHR Tables

In [6]:
# ehr_tables_CG = build_ehr_tables(directory_CG)
# output_dir = '/Users/jerenolsen/Desktop/PGP_Harvard_data/EHRS/Complete Genomics/'
# for table in ehr_tables_CG:
#     ehr_tables_CG[table].to_csv('{output_dir}CG_{table}.tsv'.format(output_dir = output_dir, table=table), sep="\t")

# Write 23andMe EHR Tables

In [7]:
# ehr_tables_23 = build_ehr_tables(directory_23)
# output_dir = '/Users/jerenolsen/Desktop/PGP_Harvard_data/EHRS/23andMe/'
# for table in ehr_tables_23:
#     ehr_tables_23[table].to_csv('{output_dir}23_{table}.tsv'.format(output_dir = output_dir, table=table), sep="\t")

# Write Combined EHR Tables

In [8]:
def combine_all_ehrs(ehr_tables_23, ehr_tables_CG, output_dir, export = False):

    combined_ehr_tables = {'demographics': None,
                  'conditions': None,
                  'medications': None,
                  'allergies': None,
                  'procedures': None,
                  'test_results': None,
                  'immunizations': None}

    for ehr_name in combined_ehr_tables.keys():
        df_23 = ehr_tables_23[ehr_name]
        profile_ids_23 = df_23['profile_id'].values.tolist()
        profile_ids_23 = list(set(profile_ids_23))

        df_CG = ehr_tables_CG[ehr_name]
        df_CG_filtered = df_CG[df_CG['profile_id'].isin(profile_ids_23) == False]

        combined = pd.concat([df_23, df_CG_filtered])
        combined.reset_index(drop=True, inplace=True)
        

        combined_ehr_tables[ehr_name] = combined
        

    if export == True:
        for ehr_table in combined_ehr_tables.keys():
            combined_ehr_tables[ehr_table].to_csv('{output_dir}/{ehr}.csv'.format(output_dir = output_dir, ehr=ehr_table), sep = ",")

    return combined_ehr_tables

In [9]:
ehr_tables_CG = build_ehr_tables(directory_CG)
ehr_tables_23 = build_ehr_tables(directory_23)
output_dir = '/Users/jerenolsen/Desktop/PGP_Harvard_data/EHRS/All Combined'
combined_ehr_tables = combine_all_ehrs(ehr_tables_23, ehr_tables_CG, output_dir, export = True)

In [10]:
#combined_ehr_tables

# Surveys 

In [11]:
#Surveys to process:
# survey_names = ['PGP PARTICIPANT SURVEY.TSV                                                              681
# PGP TRAIT & DISEASE SURVEY 2012: CANCERS.TSV                                            543
# PGP TRAIT & DISEASE SURVEY 2012: ENDOCRINE, METABOLIC, NUTRITIONAL, AND IMMUNITY.TSV    534
# PGP TRAIT & DISEASE SURVEY 2012: BLOOD.TSV                                              527
# PGP TRAIT & DISEASE SURVEY 2012: CONGENITAL TRAITS AND ANOMALIES.TSV                    524
# PGP TRAIT & DISEASE SURVEY 2012: VISION AND HEARING.TSV                                 524
# PGP TRAIT & DISEASE SURVEY 2012: SKIN AND SUBCUTANEOUS TISSUE.TSV                       519
# PGP TRAIT & DISEASE SURVEY 2012: NERVOUS SYSTEM.TSV                                     515
# PGP TRAIT & DISEASE SURVEY 2012: DIGESTIVE SYSTEM.TSV                                   514
# PGP TRAIT & DISEASE SURVEY 2012: MUSCULOSKELETAL SYSTEM AND CONNECTIVE TISSUE.TSV       509
# PGP TRAIT & DISEASE SURVEY 2012: CIRCULATORY SYSTEM.TSV                                 505
# PGP TRAIT & DISEASE SURVEY 2012: GENITOURINARY SYSTEMS.TSV                              502
# PGP TRAIT & DISEASE SURVEY 2012: RESPIRATORY SYSTEM.TSV                                 499
# PGP BASIC PHENOTYPES SURVEY 2015.TSV                                                    299]

In [12]:
#Survey Specific Functions
def get_survey_names(directory, provider_id):
    
    paths = get_filepaths(directory, 'surveys')
    
    survey_names = set()

    for path in paths:
        for survey in os.listdir(path):
            survey = os.path.join(path, survey)
            name = survey.rsplit('/', 1)[-1]
            survey_names.add(name)

    return list(survey_names)
                          
def get_survey_type(directory, provider_id, survey_substring):
    "Survey string can be any substring contained in survey name that identifies it uniquely (ie. 'Congenital')"
    
    paths = get_filepaths(directory, 'surveys')
    #paths = remove_intersecting_profiles(paths, provider_id)
    
    survey_df = pd.DataFrame()
    for path in paths:
        profile_id = path[-16:-8]
        try:
            for survey in os.listdir(path):
                survey = os.path.join(path, survey)
                df = read_table(survey)
                df.insert(0, 'profile_id', profile_id)
                name = survey.rsplit('/', 1)[-1]
            
                if survey_substring.upper() in name.upper():
                    df.drop(labels = 'Unnamed: 0',axis = 1, inplace = True)
                    survey_df = pd.concat([survey_df, df])

        except Exception as e:
            #pass
            print(f"Exception: {e}\n")

        
    
    return survey_df

def retrieve_survey_controller(directory, provider_id, survey_substring):
    surveys = get_survey_type(directory, provider_id, survey_substring)

    return surveys

In [13]:
directory = '/Users/jerenolsen/Desktop/Exploring PGP Harvard/Complete Genomics/all-data-by-profile'
names = get_survey_names(directory, 'CG')
names = [name for name in names if 'Participant' in name or 'Trait & Disease' in name]
names

['PGP Trait & Disease Survey 2012: Respiratory System.tsv',
 'PGP Trait & Disease Survey 2012: Cancers.tsv',
 'PGP Trait & Disease Survey 2012: Vision and hearing.tsv',
 'PGP Trait & Disease Survey 2012: Genitourinary Systems.tsv',
 'PGP Trait & Disease Survey 2012: Nervous System.tsv',
 'PGP Trait & Disease Survey 2012: Congenital Traits and Anomalies.tsv',
 'PGP Trait & Disease Survey 2012: Blood.tsv',
 'PGP Trait & Disease Survey 2012: Endocrine, Metabolic, Nutritional, and Immunity.tsv',
 'PGP Trait & Disease Survey 2012: Digestive System.tsv',
 'PGP Trait & Disease Survey 2012: Musculoskeletal System and Connective Tissue.tsv',
 'PGP Trait & Disease Survey 2012: Skin and Subcutaneous Tissue.tsv',
 'PGP Participant Survey.tsv',
 'PGP Trait & Disease Survey 2012: Circulatory System.tsv']

In [52]:
def init_surveys_dict():
    surveys_dict = {'Participant Survey': pd.DataFrame(),
                   'Respiratory': pd.DataFrame(),
                   'Nervous': pd.DataFrame(),
                   'Musculoskeletal': pd.DataFrame(),
                   'Vision and hearing': pd.DataFrame(),
                   'Congenital Traits': pd.DataFrame(),
                   'Endocrine, Metabolic': pd.DataFrame(),
                   'Blood': pd.DataFrame(),
                   'Skin': pd.DataFrame(),
                    'Circulatory': pd.DataFrame(),
                   'Digestive': pd.DataFrame(),
                   'Genitourinary': pd.DataFrame(),
                   'Cancers': pd.DataFrame()
                  }
    return surveys_dict

def fix_survey_cols(df):
    df.columns = df.columns.droplevel(0)
    cols = df.columns.tolist()
    cols[0] = 'profile_id'
    cols[1] = 'provider_id'
    df.columns = cols
    return df

def expand_response_col(df):
    #Splits entries in each cell of response column on commas and expands dataframe
    all_cols = df.columns
    response_col = None
    for i in range(0,len(all_cols)):
        cur_col = all_cols[i]
        if "the following conditions" in cur_col.lower():
            response_col = cur_col
            break

    df.rename(columns = {response_col:'response'}, inplace = True)

    #Remove rows containing null in response
    df.fillna(" ", inplace=True)
    df = df[df['response'] != " "]
    
    #Explode column and rename
    df = df.assign(expanded = df.response.str.split(",")).explode('expanded')
    df.drop(labels = 'response', axis = 1, inplace = True)
    df.rename(columns = {'expanded':'response'}, inplace = True)
    
    #Reposition columns
    cols = df.columns.tolist()
    expanded = [cols.pop()]
    df = df[cols[0:2]+expanded+cols[2:]]
    df.reset_index(drop = True, inplace = True)
    
    #Remove leading spaces
    for i, row in df.iterrows():
        try:
            value = row['response']
            if type(value) == str and value[0] == " ":
                df.iloc[i]['response'] = value[1:]
        except Exception as e:
            print(f"Error : {e}")
            
    return df

def read_write_survey_tables(input_dir, output_dir, provider_id, export = False):
    all_surveys = init_surveys_dict()

    for survey in all_surveys.keys():
        print(f"Processing {survey} - {provider_id}")
        survey_table = retrieve_survey_controller(input_dir, provider_id, survey.lower())
        survey_table_p = pd.pivot(survey_table, values = ['1'], columns = ['0'], index = ['profile_id'])
        survey_table_p = survey_table_p.reset_index()
        survey_table_p.insert(1, 'provider_id', [provider_id]*len(survey_table_p))#add provider id label 
        survey_table_p = fix_survey_cols(survey_table_p)
        
        
        #Expand response columns if applicable for current survey
        if "Participant Survey" not in survey:
            survey_table_p = expand_response_col(survey_table_p)
        
        all_surveys[survey] = survey_table_p
        
    if export == True:
        for survey in all_surveys.keys():
            all_surveys[survey].to_csv('{output_dir}/{p_id}_{survey}.csv'.format(output_dir = output_dir, p_id = provider_id, survey=survey), sep = ",")        
        
    print("- Done -")
    return all_surveys
    
    

# Write Complete Genomics Survey Tables

In [53]:
def write_surveys_CG(export):
    directory_CG = '/Users/jerenolsen/Desktop/Exploring PGP Harvard/Complete Genomics/all-data-by-profile'
    output_dir = '/Users/jerenolsen/Desktop/PGP_Harvard_data/Surveys/Complete Genomics'
    all_surveys_CG = read_write_survey_tables(input_dir = directory_CG, output_dir = output_dir , provider_id = 'CG', export=export)
    return all_surveys_CG 

In [54]:
all_surveys_CG = write_surveys_CG(export = True)

Processing Participant Survey - CG
Processing Respiratory - CG
Processing Nervous - CG
Processing Musculoskeletal - CG
Processing Vision and hearing - CG
Processing Congenital Traits - CG
Processing Endocrine, Metabolic - CG
Processing Blood - CG
Processing Skin - CG
Processing Circulatory - CG
Processing Digestive - CG
Processing Genitourinary - CG
Processing Cancers - CG
- Done -


# Write 23andMe Survey Tables

In [55]:
def write_surveys_23andMe(export):
    directory_23 = '/Users/jerenolsen/Desktop/Exploring PGP Harvard/23andMe/all-data-by-profile'
    output_dir = '/Users/jerenolsen/Desktop/PGP_Harvard_data/Surveys/23andMe'
    all_surveys_23 = read_write_survey_tables(input_dir = directory_23, output_dir = output_dir , provider_id = '23andMe', export=export)
    return all_surveys_23

In [56]:
all_surveys_23 = write_surveys_23andMe(export = True)

Processing Participant Survey - 23andMe
Processing Respiratory - 23andMe
Processing Nervous - 23andMe
Processing Musculoskeletal - 23andMe
Processing Vision and hearing - 23andMe
Processing Congenital Traits - 23andMe
Processing Endocrine, Metabolic - 23andMe
Processing Blood - 23andMe
Processing Skin - 23andMe
Processing Circulatory - 23andMe
Processing Digestive - 23andMe
Processing Genitourinary - 23andMe
Processing Cancers - 23andMe
- Done -


# Write Combined Surveys (CG + 23andMe)
- If Duplicate profile_id is present, remove from CG and keep 23andMe

In [57]:
def combine_all_surveys(all_surveys_23, all_surveys_CG, output_dir, export = False):

    all_surveys = init_surveys_dict()

    for s_name in all_surveys.keys():
        df_23 = all_surveys_23[s_name]
        profile_ids_23 = df_23['profile_id'].values.tolist()
        profile_ids_23 = list(set(profile_ids_23))

        df_CG = all_surveys_CG[s_name]
        df_CG_filtered = df_CG[df_CG['profile_id'].isin(profile_ids_23) == False]

        combined = pd.concat([df_23, df_CG_filtered])
        combined.reset_index(drop=True,inplace=True)

        all_surveys[s_name] = combined

    if export == True:
        for survey in all_surveys.keys():
            all_surveys[survey].to_csv('{output_dir}/{survey}.csv'.format(output_dir = output_dir, survey=survey), sep = ",")

    return all_surveys


In [58]:
#all_surveys_23 = write_surveys_23andMe(export = False)
#all_surveys_CG = write_surveys_CG(export = False)

In [59]:
output_dir = '/Users/jerenolsen/Desktop/PGP_Harvard_data/Surveys/All Combined'
all_surveys = combine_all_surveys(all_surveys_23, all_surveys_CG, output_dir = output_dir, export = True)

In [61]:
test = all_surveys['Endocrine, Metabolic']

In [63]:
test.head()

,profile_id,provider_id,response,Other condition not listed here?,"PGP Trait & Disease Survey 2012: Endocrine, Metabolic, Nutritional, and Immunity",Timestamp
0,hu005023,23andMe,Lactose intolerance,,Responses submitted 8/4/2013 21:47:10. Show r...,8/4/2013 21:47:10
1,hu016B28,23andMe,Diabetes mellitus,,Responses submitted 10/25/2012 13:13:44. Show...,10/25/2012 13:13:44
2,hu016B28,23andMe,type 2,,Responses submitted 10/25/2012 13:13:44. Show...,10/25/2012 13:13:44
3,hu016B28,23andMe,High cholesterol (hypercholesterolemia),,Responses submitted 10/25/2012 13:13:44. Show...,10/25/2012 13:13:44
4,hu016B28,23andMe,High triglycerides (hypertriglyceridemia),,Responses submitted 10/25/2012 13:13:44. Show...,10/25/2012 13:13:44
